In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sys
import gc

In [2]:
customer = pd.read_csv('input/original/Customer.csv')
customer.columns = ['CustomerIdx', 'Customer_Sector', 'Customer_Subsector', 'Customer_Region', 'Customer_Country']
print(customer.shape)
customer.head()

(3471, 5)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA
3,836,Asset Owners,Insurance,Americas,BERMUDA
4,816,Asset Owners,Insurance,Americas,BERMUDA


In [3]:
full = pd.read_csv('input/generated/trade_challenge_final.csv')
full.head()

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016


In [4]:
full.isnull().sum()

BuySell                   0
CustomerIdx               0
CustomerInterest     484758
IsinIdx                   0
Month                     0
NotionalEUR         5207872
PredictionIdx             0
Price               5270744
TimeHash                  0
WeekID                    0
Week_of_Month             0
Year                      0
dtype: int64

In [5]:
full['Prev_WeekID'] = full.WeekID-1

In [7]:
# Customer number of transactions in last week
feat = ['CustomerIdx', 'IsinIdx', 'WeekID', 'Prev_WeekID']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat].groupby(['CustomerIdx', 'WeekID', 'Prev_WeekID']).size().reset_index()
temp.columns = ['CustomerIdx', 'WeekID', 'Prev_WeekID', 'nBonds_lastWeek']
temp = temp[['CustomerIdx', 'Prev_WeekID']].merge(temp[['CustomerIdx', 'WeekID', 'nBonds_lastWeek']], left_on=['CustomerIdx', 'Prev_WeekID'], right_on=['CustomerIdx', 'WeekID'])
temp.drop('WeekID', axis=1, inplace=True)
full = full.merge(temp, how='left')
full['nBonds_lastWeek'].fillna(0, inplace=True)
del temp; gc.collect()
print(full.shape)
full.head()

(7581011, 14)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year,Prev_WeekID,nBonds_lastWeek
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016,56,13.0
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016,56,111.0
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016,56,111.0
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0


In [8]:
# Customer number of transactions in last week
feat = ['CustomerIdx', 'IsinIdx', 'WeekID', 'Prev_WeekID']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat].drop_duplicates().groupby(['CustomerIdx', 'WeekID', 'Prev_WeekID']).size().reset_index()
temp.columns = ['CustomerIdx', 'WeekID', 'Prev_WeekID', 'nUniqueBonds_lastWeek']
temp = temp[['CustomerIdx', 'Prev_WeekID']].merge(temp[['CustomerIdx', 'WeekID', 'nUniqueBonds_lastWeek']], left_on=['CustomerIdx', 'Prev_WeekID'], right_on=['CustomerIdx', 'WeekID'])
temp.drop('WeekID', axis=1, inplace=True)
full = full.merge(temp, how='left')
full['nUniqueBonds_lastWeek'].fillna(0, inplace=True)
del temp; gc.collect()
print(full.shape)
full.head()

(7581011, 15)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year,Prev_WeekID,nBonds_lastWeek,nUniqueBonds_lastWeek
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016,56,13.0,11.0
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016,56,111.0,92.0
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016,56,111.0,92.0
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0


In [9]:
full['Ratio_UniqueBonds_lastWeek'] = full.nUniqueBonds_lastWeek/full.nBonds_lastWeek
full['Ratio_UniqueBonds_lastWeek'].fillna(0, inplace=True)
full.head()

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year,Prev_WeekID,nBonds_lastWeek,nUniqueBonds_lastWeek,Ratio_UniqueBonds_lastWeek
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016,56,13.0,11.0,0.846154
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016,56,111.0,92.0,0.828829
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016,56,111.0,92.0,0.828829
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0,0.858726
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0,0.858726


In [10]:
full[['nBonds_lastWeek', 'nUniqueBonds_lastWeek', 'Ratio_UniqueBonds_lastWeek']].describe()

,nBonds_lastWeek,nUniqueBonds_lastWeek,Ratio_UniqueBonds_lastWeek
count,7.581011e+06,7.581011e+06,7.581011e+06
mean,1.165747e+02,8.014228e+01,6.055694e-01
std,2.256156e+02,1.435597e+02,3.628442e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,3.576642e-01
50%,2.300000e+01,1.800000e+01,7.500000e-01
75%,1.140000e+02,8.800000e+01,8.777778e-01
max,3.814000e+03,2.364000e+03,1.000000e+00


In [11]:
# Customer Weeks since previous trade
feat = ['CustomerIdx', 'WeekID']
a = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat]
a = a.drop_duplicates().reset_index(drop=True)
a = a.sort_values(feat).reset_index(drop=True)
a['Cust_Previous_WeekID'] = a.groupby(['CustomerIdx'])['WeekID'].shift()
a['Weeks_since_previous_trade'] = a.WeekID - a.Cust_Previous_WeekID
a['Weeks_since_previous_trade'].fillna(0, inplace=True)
a.Cust_Previous_WeekID.fillna(a.WeekID, inplace = True)
a.reset_index(inplace = True, drop =True)
full = full.merge(a, how='left', on=['CustomerIdx', 'WeekID'])
full['Weeks_since_previous_trade'].fillna(0, inplace=True)
del a; gc.collect()
print(full.shape)
full.head()

(7581011, 18)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year,Prev_WeekID,nBonds_lastWeek,nUniqueBonds_lastWeek,Ratio_UniqueBonds_lastWeek,Cust_Previous_WeekID,Weeks_since_previous_trade
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016,56,13.0,11.0,0.846154,56.0,1.0
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016,56,111.0,92.0,0.828829,56.0,1.0
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016,56,111.0,92.0,0.828829,56.0,1.0
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0,0.858726,56.0,1.0
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016,56,361.0,310.0,0.858726,56.0,1.0


In [12]:
full.columns

Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'Prev_WeekID', 'nBonds_lastWeek',
       'nUniqueBonds_lastWeek', 'Ratio_UniqueBonds_lastWeek',
       'Cust_Previous_WeekID', 'Weeks_since_previous_trade'],
      dtype='object')

In [13]:
feat = ['BuySell', 'CustomerIdx', 'IsinIdx', 'WeekID', 'nBonds_lastWeek', 'nUniqueBonds_lastWeek', 
        'Ratio_UniqueBonds_lastWeek', 'Cust_Previous_WeekID', 'Weeks_since_previous_trade']
full[feat].to_csv('input/generated/cust_week_feat.csv', index=False)

In [14]:
# Customer number of unique bonds bought
feat = ['CustomerIdx', 'IsinIdx']
temp = full[full.CustomerInterest == 1][feat].drop_duplicates().groupby(['CustomerIdx']).size().reset_index()
temp.columns = ['CustomerIdx', 'nCustUniqueBonds']
temp['nCustUniqueBonds'].fillna(0, inplace=True)
customer = customer.merge(temp, how='left', on='CustomerIdx')
print(customer.shape)
del temp; gc.collect()
customer.head()

(3471, 6)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country,nCustUniqueBonds
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS,5.0
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA,9.0
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA,13.0
3,836,Asset Owners,Insurance,Americas,BERMUDA,1.0
4,816,Asset Owners,Insurance,Americas,BERMUDA,22.0


In [15]:
# Average & Std of NotionalEUR and Price for customer
feat = ['CustomerIdx', 'NotionalEUR', 'Price']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx != 'valid')][feat].copy()

temp1 = temp.groupby(['CustomerIdx'])['NotionalEUR'].mean().reset_index()
temp1.rename(columns = {'NotionalEUR': 'Cust_NotionalEUR_Mean'}, inplace=True)
temp1['Cust_NotionalEUR_std'] = temp.groupby(['CustomerIdx'])['NotionalEUR'].std()
temp1['Cust_NotionalEUR_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

temp1 = temp.groupby(['CustomerIdx'])['Price'].mean().reset_index()
temp1.rename(columns = {'Price': 'Cust_Price_Mean'}, inplace=True)
temp1['Cust_Price_std'] = temp.groupby(['CustomerIdx'])['NotionalEUR'].std()
temp1['Cust_Price_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

print(customer.shape)
del temp1, temp; gc.collect()
customer.head()

(3471, 10)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country,nCustUniqueBonds,Cust_NotionalEUR_Mean,Cust_NotionalEUR_std,Cust_Price_Mean,Cust_Price_std
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS,5.0,3.093697e+06,6.047403e+08,132.730444,6.047403e+08
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA,9.0,2.988462e+06,1.250000e+05,102.898462,1.250000e+05
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA,13.0,1.405035e+06,1.293098e+06,102.278897,1.293098e+06
3,836,Asset Owners,Insurance,Americas,BERMUDA,1.0,1.378863e+06,1.787841e+06,98.270000,1.787841e+06
4,816,Asset Owners,Insurance,Americas,BERMUDA,22.0,5.666658e+06,1.129202e+06,103.473047,1.129202e+06


In [33]:
# Average & Std of NotionalEUR and Price for customer with Buy and sell
feat = ['CustomerIdx', 'BuySell', 'NotionalEUR', 'Price']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx != 'valid')][feat].copy()

temp1 = temp.groupby(['CustomerIdx', 'BuySell'])['NotionalEUR'].mean().reset_index()
temp1 = temp1.pivot(index='CustomerIdx', columns='BuySell', values='NotionalEUR').reset_index().fillna(0)
temp1.rename(columns = {'Buy': 'Cust_Buy_NotionalEUR_Mean', 'Sell': 'Cust_Sell_NotionalEUR_Mean'}, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

temp1 = temp.groupby(['CustomerIdx', 'BuySell'])['Price'].mean().reset_index()
temp1 = temp1.pivot(index='CustomerIdx', columns='BuySell', values='Price').reset_index().fillna(0)
temp1.rename(columns = {'Buy': 'Cust_Buy_Price_Mean', 'Sell': 'Cust_Sell_Price_Mean'}, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

print(customer.shape)
del temp1, temp; gc.collect()
customer.head()

(3471, 14)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country,nCustUniqueBonds,Cust_NotionalEUR_Mean,Cust_NotionalEUR_std,Cust_Price_Mean,Cust_Price_std,Cust_Buy_NotionalEUR_Mean,Cust_Sell_NotionalEUR_Mean,Cust_Buy_Price_Mean,Cust_Sell_Price_Mean
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS,5.0,3.093697e+06,6.047403e+08,132.730444,6.047403e+08,2.563787e+06,3.517625e+06,106.625500,153.614400
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA,9.0,2.988462e+06,1.250000e+05,102.898462,1.250000e+05,0.000000e+00,2.988462e+06,0.000000,102.898462
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA,13.0,1.405035e+06,1.293098e+06,102.278897,1.293098e+06,1.310043e+06,1.539607e+06,102.632471,101.778000
3,836,Asset Owners,Insurance,Americas,BERMUDA,1.0,1.378863e+06,1.787841e+06,98.270000,1.787841e+06,0.000000e+00,1.378863e+06,0.000000,98.270000
4,816,Asset Owners,Insurance,Americas,BERMUDA,22.0,5.666658e+06,1.129202e+06,103.473047,1.129202e+06,5.973180e+06,5.342107e+06,103.960800,102.985294


In [34]:
# Cust Avg Weeks and std between transaction
feat = ['CustomerIdx', 'Weeks_since_previous_trade']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat].copy()
temp1 = temp.groupby(['CustomerIdx'])['Weeks_since_previous_trade'].mean().reset_index()
temp1.rename(columns = {'Weeks_since_previous_trade': 'Avg_Cust_Weeks_since_previous_trade'}, inplace=True)
temp1['Cust_Weeks_since_previous_trade_std'] = temp.groupby(['CustomerIdx'])['Weeks_since_previous_trade'].std()
temp1['Cust_Weeks_since_previous_trade_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')
del feat, temp, temp1; gc.collect()
customer.head()

,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country,nCustUniqueBonds,Cust_NotionalEUR_Mean,Cust_NotionalEUR_std,Cust_Price_Mean,Cust_Price_std,Cust_Buy_NotionalEUR_Mean,Cust_Sell_NotionalEUR_Mean,Cust_Buy_Price_Mean,Cust_Sell_Price_Mean,Avg_Cust_Weeks_since_previous_trade,Cust_Weeks_since_previous_trade_std
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS,5.0,3.093697e+06,6.047403e+08,132.730444,6.047403e+08,2.563787e+06,3.517625e+06,106.625500,153.614400,3.315789,2.352267
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA,9.0,2.988462e+06,1.250000e+05,102.898462,1.250000e+05,0.000000e+00,2.988462e+06,0.000000,102.898462,8.967742,14.840822
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA,13.0,1.405035e+06,1.293098e+06,102.278897,1.293098e+06,1.310043e+06,1.539607e+06,102.632471,101.778000,1.883721,9.204633
3,836,Asset Owners,Insurance,Americas,BERMUDA,1.0,1.378863e+06,1.787841e+06,98.270000,1.787841e+06,0.000000e+00,1.378863e+06,0.000000,98.270000,5.000000,8.948571
4,816,Asset Owners,Insurance,Americas,BERMUDA,22.0,5.666658e+06,1.129202e+06,103.473047,1.129202e+06,5.973180e+06,5.342107e+06,103.960800,102.985294,6.146341,0.597809


In [ ]:
# Cust previous NotionalEur and Price: Similar as above

In [35]:
# Customer Avg bonds bought in a week
feat = ['CustomerIdx', 'WeekID', 'nBonds_lastWeek', 'nUniqueBonds_lastWeek', 'Ratio_UniqueBonds_lastWeek']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat].copy()
temp.drop_duplicates(['CustomerIdx', 'WeekID'], inplace=True)

temp1 = temp.groupby(['CustomerIdx'])['nBonds_lastWeek'].mean().reset_index()
temp1.rename(columns = {'nBonds_lastWeek': 'Cust_nBonds_Avg'}, inplace=True)
temp1['Cust_nBonds_std'] = temp.groupby(['CustomerIdx'])['nBonds_lastWeek'].std()
temp1['Cust_nBonds_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

temp1 = temp.groupby(['CustomerIdx'])['nUniqueBonds_lastWeek'].mean().reset_index()
temp1.rename(columns = {'nUniqueBonds_lastWeek': 'Cust_nUniqueBonds_Avg'}, inplace=True)
temp1['Cust_nUniqueBonds_std'] = temp.groupby(['CustomerIdx'])['nUniqueBonds_lastWeek'].std()
temp1['Cust_nUniqueBonds_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

temp1 = temp.groupby(['CustomerIdx'])['Ratio_UniqueBonds_lastWeek'].mean().reset_index()
temp1.rename(columns = {'Ratio_UniqueBonds_lastWeek': 'Cust_ratioUniqueBonds_Avg'}, inplace=True)
temp1['Cust_ratioUniqueBonds_std'] = temp.groupby(['CustomerIdx'])['Ratio_UniqueBonds_lastWeek'].std()
temp1['Cust_ratioUniqueBonds_std'].fillna(0, inplace=True)
customer = customer.merge(temp1, how='left', on='CustomerIdx')

customer['Cust_ratio_Avg_nBonds'] = customer.Cust_nUniqueBonds_Avg/customer.Cust_nBonds_Avg
customer['Cust_ratio_Avg_nBonds'].fillna(0, inplace=True)

print(customer.shape)
del temp1, temp; gc.collect()
customer.head()

(3471, 23)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country,nCustUniqueBonds,Cust_NotionalEUR_Mean,Cust_NotionalEUR_std,Cust_Price_Mean,Cust_Price_std,...,Cust_Sell_Price_Mean,Avg_Cust_Weeks_since_previous_trade,Cust_Weeks_since_previous_trade_std,Cust_nBonds_Avg,Cust_nBonds_std,Cust_nUniqueBonds_Avg,Cust_nUniqueBonds_std,Cust_ratioUniqueBonds_Avg,Cust_ratioUniqueBonds_std,Cust_ratio_Avg_nBonds
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS,5.0,3.093697e+06,6.047403e+08,132.730444,6.047403e+08,...,153.614400,3.315789,2.352267,0.000000,0.484612,0.000000,0.484612,0.000000,0.391675,0.000
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA,9.0,2.988462e+06,1.250000e+05,102.898462,1.250000e+05,...,102.898462,8.967742,14.840822,2.500000,1.154701,1.750000,0.577350,0.350000,0.288675,0.700
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA,13.0,1.405035e+06,1.293098e+06,102.278897,1.293098e+06,...,101.778000,1.883721,9.204633,0.714286,0.744024,0.571429,0.744024,0.428571,0.462910,0.800
3,836,Asset Owners,Insurance,Americas,BERMUDA,1.0,1.378863e+06,1.787841e+06,98.270000,1.787841e+06,...,98.270000,5.000000,8.948571,0.000000,0.707107,0.000000,0.707107,0.000000,0.440959,0.000
4,816,Asset Owners,Insurance,Americas,BERMUDA,22.0,5.666658e+06,1.129202e+06,103.473047,1.129202e+06,...,102.985294,6.146341,0.597809,0.363636,3.562372,0.318182,2.564688,0.295455,0.411153,0.875


In [ ]:
# Weeks the customer has been active: last transaction - first


In [ ]:
# Do the user buy the most popular bond: Binary

In [36]:
customer.to_csv('input/generated/customer_feat.csv', index=False)

In [37]:
customer.isnull().sum()

CustomerIdx                              0
Customer_Sector                          0
Customer_Subsector                     352
Customer_Region                          0
Customer_Country                         0
nCustUniqueBonds                        32
Cust_NotionalEUR_Mean                   32
Cust_NotionalEUR_std                    32
Cust_Price_Mean                         94
Cust_Price_std                          32
Cust_Buy_NotionalEUR_Mean               32
Cust_Sell_NotionalEUR_Mean              32
Cust_Buy_Price_Mean                     32
Cust_Sell_Price_Mean                    32
Avg_Cust_Weeks_since_previous_trade     32
Cust_Weeks_since_previous_trade_std     32
Cust_nBonds_Avg                         32
Cust_nBonds_std                         32
Cust_nUniqueBonds_Avg                   32
Cust_nUniqueBonds_std                   32
Cust_ratioUniqueBonds_Avg               32
Cust_ratioUniqueBonds_std               32
Cust_ratio_Avg_nBonds                    0
dtype: int6

In [38]:
full.isnull().sum()

BuySell                             0
CustomerIdx                         0
CustomerInterest               484758
IsinIdx                             0
Month                               0
NotionalEUR                   5207872
PredictionIdx                       0
Price                         5270744
TimeHash                            0
WeekID                              0
Week_of_Month                       0
Year                                0
Prev_WeekID                         0
nBonds_lastWeek                     0
nUniqueBonds_lastWeek               0
Ratio_UniqueBonds_lastWeek          0
Cust_Previous_WeekID          1159594
Weeks_since_previous_trade          0
dtype: int64